Load packages

In [1]:
import os
os.chdir('../../')

In [2]:
cd = os.getcwd()
functions_path = os.path.join(cd, '02_code/functions')

In [3]:
import scanpy as sc
import scanpy.external as sce
import numpy as np
import pandas as pd
import anndata as ad
import seaborn as sns
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix
from anndata import AnnData
import skmisc
import regex as re

import anndata2ri
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

# from scipy.sparse import csr_matrix
import sys
sys.path.append(functions_path)
import functions_dat_processing as dp
import functions_plotting as dplt
import functions_for_CAR_annotation as ca

/tmp/ipykernel_578777/1078855118.py:19: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [19]:
#if needed, reload functions
import importlib
importlib.reload(dp)
importlib.reload(dplt)
importlib.reload(ca)

<module 'functions_for_CAR_annotation' from '/home/s377963/car_t_sc/02_code/functions/functions_for_CAR_annotation.py'>

Import data

In [147]:
adata_raw = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_pools_prior_any_processing.h5ad') #prior to any sort of processing, shows maximal possible yield with respect to CAR-T annotation
adata_dLN = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_calagry_exact_TIL_only.h5ad') #after dLN filtering
adata_pureTC = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_1xintegrated_pureTCs_calagry_exact.h5ad') #after annotating pureTCs and prior to anotating TC subtypes
adata_pure_TC_and_rest = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_calagry_exact_TIL_only_celltypes.h5ad') #after annotating pureTCs and prior to anotating TC subtypes
adata_annotated = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_1xintegrated_scaled_pureTCs_annotated_calagry_exact.h5ad') #after annotating TCs and TC subtypes

# differently processed datasets
# adata = sc.read_h5ad('/home/s377963/car_t_sc/01_data/processed/merged_and_processed/merged_1xintegrated_scaled_pureTCs_annotated_pseudocount.h5ad')
# adata = sc.read('./01_data/processed/merged_and_processed/hashsolo_abs_integrated_hvg_TCannotated_TCsubtypes.h5ad')
# adata = sc.read('./01_data/processed/merged_and_processed/hashsolo_mad_soupx_integrated_hvg_TCannotated_TCsubtypes.h5ad')
# adata = sc.read('./01_data/processed/merged_and_processed/hashsolo_mad_integrated_hvg_TCannotated_TCsubtypes.h5ad')
# adata = sc.read('./01_data/processed/merged_and_processed/HTODemux_abs_1000_nosoup.h5ad')

In [148]:
for adata in [adata_raw, adata_dLN, adata_pure_TC_and_rest, adata_pureTC, adata_annotated]:
    adata.obs.rename(columns={"HTO_classification": "Classification"}, inplace=True) 
    adata.obs.rename(columns={"functional.cluster": "Tcell_subtype"}, inplace=True)
    adata.obs.rename(columns={"pool": "dataset"}, inplace=True)
    rmvd_p = [int(dataset.replace("P", "")) - 1 for dataset in adata.obs['dataset']]
    adata.obs['dataset'] = rmvd_p

In [149]:
#only needed for adata_annotated, since it still contains NAs for TC subtypes
nas = adata_annotated.obs.Tcell_subtype == "NA"
adata_annotated_nona = adata_annotated[~nas,:].copy()

In [150]:
path = './01_data/processed/count_cart_receptor'
VDJ_GEX_list = ca.read_and_merge_CAR_annotation(path) #list with dims: list[1][8]
merged = [ca.merge_VDJ_and_GEX(GEX, VDJ) for (GEX, VDJ) in zip(*VDJ_GEX_list)]

annotated = []
for adata in [adata_raw, adata_dLN, adata_pure_TC_and_rest, adata_pureTC, adata_annotated]:
    new_adata = ca.annotate_mapped_cars(adata, merged)
    annotated.append(new_adata)

/home/s377963/car_t_sc/02_code/functions/functions_for_CAR_annotation.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged.fillna(0, inplace=True)
/home/s377963/car_t_sc/02_code/functions/functions_for_CAR_annotation.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged.fillna(0, inplace=True)
/home/s377963/car_t_sc/02_code/functions/functions_for_CAR_annotation.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To 

In [139]:
#check what celltypes CAR TCR containing cells are assigned to if not pure TCs
car_and_celltypes_annotated = annotated[2]
only_car = car_and_celltypes_annotated[car_and_celltypes_annotated.obs.isCAR == True]
only_car_nonTCs = only_car[only_car.obs['is.pure_Tcell'] == 'Impure']

In [132]:
#see that cell not assigned as pureTCs that have reads matching the CD19 CAR-T marker are not assigned to any cell type at all (probably due to lacking marker genes)
only_car_nonTCs.obs.scGate_multi.unique()

[NaN]
Categories (0, object): []

In [155]:
#for raw (theoretical maximum of annotated CAR-Ts)
only_cars0 = annotated[0][annotated[0].obs.isCAR]
cars_beginning = only_cars0.obs.groupby('dataset').size()
cars_beginning

dataset
0    5169
1      24
2      12
3      13
4      11
5       3
6      15
7      10
8       4
dtype: int64

In [178]:
#for only TILs after qc and demultiplexing
only_cars1 = annotated[1][annotated[1].obs.isCAR]
cars_qc = only_cars1.obs.groupby('dataset').size()

pools = range(0,9)
cars_qc = cars_qc.reindex(pools, fill_value=0)
cars_qc

dataset
0    1177
1       4
2       8
3       0
4       3
5       0
6      10
7       0
8       1
dtype: int64

In [180]:
#whats lost due to demultiplexing, qc and eliminating dLNs
(cars_beginning - cars_qc) / cars_beginning *100

dataset
0     77.229638
1     83.333333
2     33.333333
3    100.000000
4     72.727273
5    100.000000
6     33.333333
7    100.000000
8     75.000000
dtype: float64

In [182]:
#for after annotation
only_cars2 = annotated[3][annotated[3].obs.isCAR]
cars_tcan = only_cars2.obs.groupby('dataset').size()
cars_tcan = cars_tcan.reindex(pools, fill_value=0)
cars_tcan

dataset
0    1176
1       1
2       0
3       0
4       1
5       0
6       1
7       0
8       0
dtype: int64

In [186]:
#whats lost due to annotating celltypes with scGate
(cars_qc - cars_tcan) / cars_beginning *100

dataset
0     0.019346
1    12.500000
2    66.666667
3     0.000000
4    18.181818
5     0.000000
6    60.000000
7     0.000000
8    25.000000
dtype: float64

In [187]:
#for after annotation
only_cars3 = annotated[4][annotated[4].obs.isCAR]
end_cars = only_cars3.obs.groupby('dataset').size()
end_cars = end_cars.reindex(pools, fill_value=0)
end_cars

dataset
0    1176
1       1
2       0
3       0
4       1
5       0
6       1
7       0
8       0
dtype: int64

In [206]:
#awnser question as to how many carTs are detected as CAR-Ts in the course of the experiment with respect to pool1, since those cells should all be cart cells
number_cells_after_qc = annotated[1][annotated[1].obs.dataset == 0].shape[0]
number_CAR_after_qc = only_cars1.shape[0]

number_CAR_after_qc / number_cells_after_qc

0.7462779156327544

In [210]:
number_cells_beginning = annotated[0][annotated[0].obs.dataset == 0].shape[0]
number_CAR_beginning = only_cars0.shape[0]

number_CAR_beginning / number_cells_beginning

0.5830008865248227